# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# use pandas function to read in csv from WeatherPy output data folder
cities_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,laguna,38.4210,-121.4238,48.06,39.0,1.0,1.34,US,1.629948e+09
1,vaini,-21.2000,-175.2000,44.27,64.0,20.0,6.69,TO,1.629948e+09
2,tessalit,20.1986,1.0114,47.52,51.0,9.0,4.29,ML,1.629948e+09
3,khatanga,71.9667,102.5000,33.98,73.0,100.0,5.91,RU,1.629948e+09
4,busselton,-33.6500,115.3333,40.66,76.0,77.0,10.72,AU,1.629948e+09
...,...,...,...,...,...,...,...,...,...
533,honiara,-9.4333,159.9500,48.58,67.0,70.0,4.21,SB,1.629948e+09
534,iralaya,15.0000,-83.2333,46.74,82.0,99.0,4.34,HN,1.629948e+09
535,constitucion,-35.3333,-72.4167,36.89,86.0,99.0,1.90,CL,1.629948e+09
536,cayenne,4.9333,-52.3333,47.15,81.0,0.0,0.45,GF,1.629948e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# use stored google api-key to 
gmaps.configure(api_key=g_key)

In [10]:
# pull latitude, longitude, and humidity into lists
lat = cities_df['Lat'].tolist()
lng = cities_df['Lng'].tolist()
#humidity = cities_df['Humidity'].tolist()

# create list of locations
indexval = np.arange(0, 538, 1)
locations = [(float(lat[ind]), float(lng[ind])) for ind in indexval]
humidity = [float(pct) for pct in cities_df['Humidity']]

In [31]:
fig = gmaps.figure(layout={'width': '800px', 'height': '500px'}, center= (0,0), zoom_level=2)

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity)

#
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='500px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
